1. 네이버 증권 "시가총액" 페이지에 있는 모든 코스피 주식과 코스닥 주식을 수집하세요.
2. 코드를 실행했을 때, 제공된 파일과 같은 파일이 한번에 자동으로 생성되는 코드를 작성하세요.

In [ ]:
# # 처음 작성 때 wt
# f = open('test.txt', 'wt')
# f.write('hello\nhello')
# f.write('\n')
# f.close

# # 추가 작성 때 'at'
# f = open('test.txt', 'at')
# f.write('hello\nhello')
# f.close

In [4]:
from datetime import datetime

str(datetime.today().date())

'2024-03-04'

In [2]:
from bs4 import BeautifulSoup
import requests
import random
import time
import pandas as pd

pi_url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page=1")
pi_html = BeautifulSoup(pi_url.text)

kospi_page = int(pi_html.find('td', class_ = "pgRR").find('a')['href'][-2:])

daq_url = requests.get("https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page=1")
daq_html = BeautifulSoup(daq_url.text)

kosdaq_page = int(daq_html.find('td', class_ = "pgRR").find('a')['href'][-2:])

print(kospi_page, kosdaq_page)

44 35


In [3]:
def make_table(html):
    table = html.find("table", class_='type_2')
    table = pd.read_html(str(table))[0]
    table = table.drop(['N','토론실'],axis=1)
    table = table[table['종목명'].notnull()]
    return table

In [6]:
# for문 진행사항 확인
from tqdm import tqdm

kospi_box=[]
kosdaq_box = []
for n in tqdm(range(1, kospi_page+1)):
    url = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=0&page={n}")
    html = BeautifulSoup(url.text)
    table = make_table(html)
    kospi_box.append(table)
    time.sleep(random.random()+0.75)
    
for n in tqdm(range(1, kosdaq_page+1)):
    url = requests.get(f"https://finance.naver.com/sise/sise_market_sum.naver?sosok=1&page={n}")
    html = BeautifulSoup(url.text)
    table = make_table(html)
    kosdaq_box.append(table)
    time.sleep(random.random()+0.75)

100%|██████████████████████████████████████████████████████████████████████████████████| 35/35 [00:56<00:00,  1.61s/it]


In [10]:
kospi = pd.concat(kospi_box, ignore_index=True)
kosdaq = pd.concat(kosdaq_box, ignore_index=True)

kospi.head(3)

,종목명,현재가,전일비,등락률,액면가,시가총액,상장주식수,외국인비율,거래량,PER,ROE
0,삼성전자,74600.0,1200.0,+1.63%,100.0,4453458.0,5969783.0,54.57,5956193.0,35.01,4.15
1,SK하이닉스,162500.0,6300.0,+4.03%,5000.0,1183004.0,728002.0,54.12,1240621.0,-10.30,3.56
2,LG에너지솔루션,405000.0,3500.0,+0.87%,500.0,947700.0,234000.0,4.93,22717.0,62.29,5.75


In [11]:
kospi.shape, kosdaq.shape

((2158, 11), (1715, 11))

In [15]:
kospi.dropna()

0        35.01
1       -10.30
2        62.29
3        64.81
4         6.16
         ...  
2120     14.70
2129    -38.31
2130    112.26
2131     39.74
2132     54.15
Name: PER, Length: 935, dtype: float64

In [38]:
from datetime import datetime

# 코스닥 기업 PER 오름차순 정리
daq_per = kosdaq[['종목명','PER']].dropna()
daq_per = daq_per.sort_values('PER')
daq_per = daq_per[daq_per['PER']>=0]

f = open("생성 파일/코스닥 기업 PER 오름차순 정리",'wt')
f.write(f"[{str(datetime.today().date())}] 코스닥 기업 PER 오름차순 정리\n\nPER(Price Earnings Ratio)은 주가수익비율을 말합니다. 이는 기업의 주가를 그 기업의 주당순이익(EPS, Earnings Per Share)으로 나눈 값으로 계산됩니다. PER은 주식이 현재 가격에 비해 얼마나 비싸거나 싼지를 평가하는 데 사용되며, 투자자들이 주식을 구매할지 여부를 결정하는 데 중요한 지표 중 하나입니다.\n\n")
f.close()

for n in range(len(daq_per)):
    f = open("생성 파일/코스닥 기업 PER 오름차순 정리",'at')
    f.write(f"{str(daq_per.iloc[n,0])} : {str(daq_per.iloc[n,1])}\n")
    f.close()

In [40]:
# 코스닥 기업 ROE 내림차순 정리
from datetime import datetime

daq_roe = kosdaq[['종목명','ROE']].dropna()
daq_roe = daq_roe.sort_values('ROE', ascending=False)
daq_roe = daq_roe[daq_roe['ROE']>=0]

f = open("생성 파일/코스닥 기업 ROE 내림차순 정리",'wt')
f.write(f"[{str(datetime.today().date())}] 코스닥 기업 ROE 내림차순 정리\n\nROE(Return on Equity)는 자기자본이익률을 말합니다. 이는 기업이 자기 자본(주주들로부터 받은 자금 및 재투자된 이익)으로 얼마나 효율적으로 이익을 창출하는지를 나타내는 지표입니다. ROE는 기업의 수익성, 자본의 효율성, 그리고 주주 가치 창출 능력을 평가하는 데 중요한 지표로 사용됩니다.\n\n")
f.close()

for n in range(len(daq_roe)):
    f = open("생성 파일/코스닥 기업 ROE 내림차순 정리",'at')
    f.write(f"{str(daq_roe.iloc[n,0])} : {str(daq_roe.iloc[n,1])}\n")
    f.close()

In [41]:
# 코스피 기업 PER 오름차순 정리
from datetime import datetime

pi_per = kospi[['종목명','PER']].dropna()
pi_per = pi_per.sort_values('PER')
pi_per = pi_per[pi_per['PER']>=0]

f = open("생성 파일/코스피 기업 PER 오름차순 정리",'wt')
f.write(f"[{str(datetime.today().date())}] 코스피 기업 PER 오름차순 정리\n\nPER(Price Earnings Ratio)은 주가수익비율을 말합니다. 이는 기업의 주가를 그 기업의 주당순이익(EPS, Earnings Per Share)으로 나눈 값으로 계산됩니다. PER은 주식이 현재 가격에 비해 얼마나 비싸거나 싼지를 평가하는 데 사용되며, 투자자들이 주식을 구매할지 여부를 결정하는 데 중요한 지표 중 하나입니다.\n\n")
f.close()

for n in range(len(pi_per)):
    f = open("생성 파일/코스피 기업 PER 오름차순 정리",'at')
    f.write(f"{str(pi_per.iloc[n,0])} : {str(pi_per.iloc[n,1])}\n")
    f.close()

In [42]:
# 코스피 기업 ROE 내림차순 정리

from datetime import datetime

pi_roe = kospi[['종목명','ROE']].dropna()
pi_roe = pi_roe.sort_values('ROE', ascending=False)
pi_roe = pi_roe[pi_roe['ROE']>=0]

f = open("생성 파일/코스닥 기업 ROE 내림차순 정리",'wt')
f.write(f"[{str(datetime.today().date())}] 코스닥 기업 ROE 내림차순 정리\n\nROE(Return on Equity)는 자기자본이익률을 말합니다. 이는 기업이 자기 자본(주주들로부터 받은 자금 및 재투자된 이익)으로 얼마나 효율적으로 이익을 창출하는지를 나타내는 지표입니다. ROE는 기업의 수익성, 자본의 효율성, 그리고 주주 가치 창출 능력을 평가하는 데 중요한 지표로 사용됩니다.\n\n")
f.close()

for n in range(len(pi_roe)):
    f = open("생성 파일/코스닥 기업 ROE 내림차순 정리",'at')
    f.write(f"{str(pi_roe.iloc[n,0])} : {str(pi_roe.iloc[n,1])}\n")
    f.close()

### 강사님

In [49]:
today = str(datetime.today().date())

sample = kospi.dropna(subset='PER').sort_values('PER')
text = f"[{today}] 코스피 기업 PER 오름차순 정리\n\n"
text += "PER(Price Earnings Ratio)은 주가수익비율을 말합니다. 이는 기업의 주가를 그 기업의 주당순이익(EPS, Earnings Per Share)으로 나눈 값으로 계산됩니다. PER은 주식이 현재 가격에 비해 얼마나 비싸거나 싼지를 평가하는 데 사용되며, 투자자들이 주식을 구매할지 여부를 결정하는 데 중요한 지표 중 하나입니다.\n\n"

for i in range(len(sample)):
    a = sample.iloc[i]
    if a['PER'] > 0:
        text += f"{a['종목명']} : {a['PER']}\n"
    
print(text)

f = open(f"생성 파일/[{today}] 코스피 기업 PER 오름차순 정리.txt",'wt')
f.write(text)
f.close()

[2024-03-04] 코스피 기업 PER 오름차순 정리

PER(Price Earnings Ratio)은 주가수익비율을 말합니다. 이는 기업의 주가를 그 기업의 주당순이익(EPS, Earnings Per Share)으로 나눈 값으로 계산됩니다. PER은 주식이 현재 가격에 비해 얼마나 비싸거나 싼지를 평가하는 데 사용되며, 투자자들이 주식을 구매할지 여부를 결정하는 데 중요한 지표 중 하나입니다.

현대지에프홀딩스 : 0.39
전방 : 0.76
넥스틸 : 0.81
태영건설 : 0.89
대성홀딩스 : 1.06
휴스틸 : 1.3
솔루스첨단소재1우 : 1.39
태영건설우 : 1.43
KG스틸 : 1.43
흥국화재 : 1.46
무학 : 1.63
삼성공조 : 1.65
세방우 : 1.67
E1 : 1.68
티웨이홀딩스 : 1.84
OCI홀딩스 : 1.85
아시아나항공 : 1.91
흥국화재우 : 1.92
이지스밸류리츠 : 2.0
팜스코 : 2.01
화천기공 : 2.01
다우기술 : 2.12
일신방직 : 2.17
평화홀딩스 : 2.19
문배철강 : 2.25
세아제강 : 2.27
솔루스첨단소재2우B : 2.42
한신공영 : 2.43
NI스틸 : 2.43
SK디스커버리우 : 2.44
KG케미칼 : 2.47
태광산업 : 2.51
한화3우B : 2.56
에어부산 : 2.57
GS우 : 2.59
한화손해보험 : 2.77
세아제강지주 : 2.79
화성산업 : 2.79
세방 : 2.79
삼양홀딩스우 : 2.8
선진 : 2.87
서연 : 2.87
한국특강 : 2.89
계룡건설 : 2.92
유성기업 : 2.98
삼양사우 : 3.0
영원무역 : 3.12
미창석유 : 3.15
한국자산신탁 : 3.22
NPC우 : 3.22
대우건설 : 3.22
영원무역홀딩스 : 3.23
동원금속 : 3.23
한국철강 : 3.3
진흥기업 : 3.32
SK디스커버리 : 3.37
현대코퍼레이션홀딩스 : 3.37
콤텍시스템 : 3.38
한국가스공사 : 3.39
GS : 3.4
SK가스 : 3.42
페이퍼코리아 : 3